In [ ]:
#需求	Sheet	子需求	功能点1	功能点2	功能点3	功能点4	功能点5	功能点6	功能点7	功能点8
#基于策略的股票自动买卖方案	Code1final	宏观经济指标预测	获取宏观经济数据	计算市场情绪指数权重(半年回测)	计算市场情绪指数	S&P 500 与情绪指数回归分析	定义指标更新时间表和API请求			
	#Code2final	股票买卖算法	数据获取与 Beta 筛选	回测与优化	资金分配与交易成本	交易信号生成与定期调整	主流程与定时任务			
	#Code3final	股票操作	股票下单函数	查询当前持仓	根据交易信号执行操作	集成到主流程				
	#Code4final	后续仓位监测和每月调整								
	#Code5final	损益评估算法	获取 Benchmark 数据	计算投资组合的收益	对比投资组合与 Benchmark 的收益	集成到主流程				


In [ ]:
#Code1final	宏观经济指标预测
import requests
import numpy as np
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from ib_insync import IB, Stock
import random
import time
import matplotlib.pyplot as plt

# 初始化 IBKR API 连接
ib = IB()
client_id = random.randint(1, 1000)
ib.connect('127.0.0.1', 7496, clientId=client_id)

# FRED API Key 和 URL
FRED_API_KEY = 'dd010e86ade20569fadc91c159648a61'
FRED_BASE_URL = 'https://api.stlouisfed.org/fred/series/observations'

# 指标配置
INDICATORS = {
    'GDP': 'GDPC1',
    'CPI': 'CPIAUCSL',
    'UNEMPLOYMENT': 'UNRATE',
    'CONSUMER_SENTIMENT': 'UMCSENT'
}

# 获取 FRED 数据
def get_fred_data(series_id, start_date):
    url = f"{FRED_BASE_URL}?series_id={series_id}&api_key={FRED_API_KEY}&file_type=json&observation_start={start_date}"
    response = requests.get(url)
    data = response.json()
    return [float(obs['value']) for obs in data.get('observations', []) if 'value' in obs]

# 使用插值法填充低频数据
def fill_to_weekly_interpolated(data, target_length):
    original_length = len(data)
    if original_length < 2:
        return data * target_length  # 如果数据不足两点，无法进行插值，直接重复填充
    x_original = np.linspace(0, target_length - 1, original_length)
    x_target = np.arange(target_length)
    interpolator = interp1d(x_original, data, kind='linear', fill_value="extrapolate")
    interpolated_data = interpolator(x_target)
    return interpolated_data

# 获取并处理宏观数据
def get_macro_data(start_date):
    all_data = {}
    target_length = 52  # 假设一年有 52 周
    for name, series_id in INDICATORS.items():
        values = get_fred_data(series_id, start_date)
        if values:
            filled_values = fill_to_weekly_interpolated(values, target_length)
            if len(filled_values) == target_length:
                standardized_values = StandardScaler().fit_transform(np.array(filled_values).reshape(-1, 1)).flatten()
                all_data[name] = standardized_values
                print(f"{name} 标准化后数据长度: {len(standardized_values)}, 内容: {standardized_values}")
                
                # 绘制填充后数据的曲线
                plt.plot(standardized_values, label=name)
            else:
                print(f"{name} 数据不足，填充后仅有 {len(filled_values)} 条记录，需 {target_length} 条。")
        else:
            print(f"{name} 无数据返回")
    plt.legend()
    plt.title("填充并标准化后的宏观经济指标数据")
    plt.xlabel("时间（周）")
    plt.ylabel("标准化值")
    plt.show()
    return all_data

    
# 从 IBKR API 获取每周 S&P 500 数据
def get_sp500_data():
    stock = Stock('SPY', 'SMART', 'USD')  # 使用 SPY 作为 S&P 500 的代表
    bars = ib.reqHistoricalData(stock, endDateTime='', durationStr='1 Y', barSizeSetting='1 week', whatToShow='TRADES', useRTH=True)
    sp500_data = [bar.close for bar in bars]
    return sp500_data

# 计算情绪指数并显示权重
def calculate_sentiment_index(data):
    if not data:
        print("缺少有效的宏观数据，无法计算情绪指数。")
        return None
    data_matrix = np.array(list(data.values())).T
    pca = PCA(n_components=1)
    principal_components = pca.fit_transform(data_matrix)
    sentiment_index = principal_components.flatten()
    
    # 显示每个指标的权重
    weights = pca.components_[0]
    indicator_names = list(data.keys())
    print("情绪指数中各经济指标的权重：")
    for name, weight in zip(indicator_names, weights):
        print(f"{name} 的权重: {weight:.4f}")
    
    return sentiment_index

# 回归分析并展示结果
def regression_analysis(sentiment_values, sp500_values):
    X = np.array(sentiment_values).reshape(-1, 1)
    y = np.array(sp500_values).reshape(-1, 1)
    model = LinearRegression()
    model.fit(X, y)
    r_squared = model.score(X, y)

    # 绘制回归分析曲线
    plt.scatter(X, y, color='blue', label='实际数据点')
    plt.plot(X, model.predict(X), color='red', label='回归线')
    plt.title(f'回归分析曲线 (R² = {r_squared:.4f})')
    plt.xlabel("情绪指数")
    plt.ylabel("S&P 500")
    plt.legend()
    plt.show()

    return r_squared

# 主流程
def main():
    # 获取过去一年的数据
    start_date = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')
    macro_data = get_macro_data(start_date)

    if len(macro_data) < 1:
        print("缺少有效的宏观数据，无法计算情绪指数。")
        return

    # 填充和标准化数据
    standardized_data = standardize_and_fill_data(macro_data, target_length=52)

    # 计算情绪指数
    sentiment_index = calculate_sentiment_index(standardized_data)

    # 获取每周 S&P 500 数据
    sp500_values = get_sp500_data()

    # 确保情绪指数和S&P 500长度一致
    min_length = min(len(sentiment_index), len(sp500_values))
    sentiment_index = sentiment_index[:min_length]
    sp500_values = sp500_values[:min_length]

    # 回归分析
    r_squared = regression_analysis(sentiment_index, sp500_values)

    # 根据回归结果判断交易策略
    if r_squared > 0.5:
        print("市场情绪有效，执行动量策略")
    else:
        print("市场情绪无效，暂停交易")

# 执行主函数
main()

# 在结束时断开 IBKR 连接
ib.disconnect()


In [ ]:
#Code2final	股票买卖算法
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from ib_insync import IB, Stock
import random

# 从CSV文件读取S&P 500成分股代码
def get_sp500_symbols_from_csv(file_path='C:\\Users\\Administrator\\Downloads\\sp500.csv'):
    df = pd.read_csv(file_path)
    df['Symbol'] = df['Symbol'].str.replace('.', ' ')
    return df['Symbol'].tolist()

# 初始化 IBKR API 连接
ib = IB()
client_id = random.randint(1, 1000)
ib.connect('127.0.0.1', 7496, clientId=client_id)

# 获取股票的历史数据
def get_historical_data(symbol, start_date, end_date):
    stock = Stock(symbol, 'SMART', 'USD')
    bars = ib.reqHistoricalData(stock, endDateTime=end_date, durationStr='180 D', barSizeSetting='1 day', whatToShow='TRADES', useRTH=True)
    df = pd.DataFrame([{'date': bar.date, 'close': bar.close} for bar in bars])
    df.set_index('date', inplace=True)
    return df

# 计算 Beta
def calculate_beta(stock_symbol, market_symbol='SPY', months=6):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=30 * months)
    stock_data = get_historical_data(stock_symbol, start_date.strftime('%Y%m%d %H:%M:%S'), end_date.strftime('%Y%m%d %H:%M:%S'))
    market_data = get_historical_data(market_symbol, start_date.strftime('%Y%m%d %H:%M:%S'), end_date.strftime('%Y%m%d %H:%M:%S'))
    combined_data = stock_data.join(market_data, lsuffix='_stock', rsuffix='_market').dropna()
    if len(combined_data) < 2:
        print(f"无法计算 {stock_symbol} 的 Beta，数据不足。")
        return None
    stock_returns = combined_data['close_stock'].pct_change().dropna()
    market_returns = combined_data['close_market'].pct_change().dropna()
    covariance = np.cov(stock_returns, market_returns)[0, 1]
    market_variance = np.var(market_returns)
    return covariance / market_variance

# 计算 RSI 指标
def compute_rsi(series, window):
    delta = series.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs)).fillna(50)

# 回测和优化策略
def backtest_strategy(df, momentum_window, sma_window, rsi_window):
    df['momentum'] = df['close'].pct_change(momentum_window)
    df['sma'] = df['close'].rolling(sma_window).mean()
    df['rsi'] = compute_rsi(df['close'], rsi_window)
    df['signal'] = np.where((df['momentum'] > 0) & (df['close'] > df['sma']) & (df['rsi'] < 70), 1, 0)
    df['returns'] = df['close'].pct_change() * df['signal'].shift(1)
    total_return = (df['returns'] + 1).prod() - 1
    sharpe_ratio = np.mean(df['returns']) / np.std(df['returns']) if df['returns'].std() != 0 else np.nan
    return total_return, sharpe_ratio

# 优化窗口期
def optimize_windows(df):
    best_params = None
    best_sharpe = -np.inf
    for momentum_window in range(10, 31, 5):
        for sma_window in range(20, 61, 10):
            for rsi_window in range(7, 21, 7):
                total_return, sharpe_ratio = backtest_strategy(df, momentum_window, sma_window, rsi_window)
                if not np.isnan(sharpe_ratio) and sharpe_ratio > best_sharpe:
                    best_sharpe = sharpe_ratio
                    best_params = (momentum_window, sma_window, rsi_window)
    return best_params

# 生成交易信号
def generate_trading_signal(df, momentum_window, beta):
    df['momentum'] = df['close'].pct_change(momentum_window)
    latest_row = df.iloc[-1]
    recommendation = "买入" if latest_row['momentum'] > 0 and latest_row['close'] > latest_row['sma'] and latest_row['rsi'] < 70 else "卖出"
    close = latest_row['close']
    momentum = latest_row['momentum']
    sma = latest_row['sma']
    rsi = latest_row['rsi']
    close_date = latest_row.name.strftime('%m%d%Y')
    comparison = '>' if close > sma else '<'
    return recommendation, close, momentum, sma, rsi, close_date, comparison

# 主流程：生成买卖信号
def generate_signals():
    signals = {}
    sp500_symbols = get_sp500_symbols_from_csv()  # 获取S&P 500成分股列表

    for symbol in sp500_symbols:
        beta = calculate_beta(symbol)
        if beta is not None and beta >= 1.0:
            end_date = datetime.now()
            start_date = end_date - timedelta(days=180)
            df = get_historical_data(symbol, start_date.strftime('%Y%m%d %H:%M:%S'), end_date.strftime('%Y%m%d %H:%M:%S'))
            best_params = optimize_windows(df)
            if best_params:
                momentum_window, sma_window, rsi_window = best_params
                recommendation, close, momentum, sma, rsi, close_date, comparison = generate_trading_signal(df, momentum_window, beta)
                signals[symbol] = {
                    "Recommendation": recommendation,
                    "Close Date": close_date,
                    "Close": close,
                    "Momentum": momentum,
                    "SMA": sma,
                    "RSI": rsi,
                    "Comparison": comparison
                }
    return signals
    
# 调用生成信号的主函数并查看结果
signals = generate_signals()

# 打印 signals 字典以查看每只股票的买入或卖出信号
print("\n生成的买卖信号：")
for symbol, data in signals.items():
    print(f"{symbol}: {data['Recommendation']} (Close on {data['Close Date']} {data['Close']:.4f} {data['Comparison']} SMA {data['SMA']:.4f}, Momentum: {data['Momentum']:.4f}, RSI: {data['RSI']:.2f})")

# 断开 IBKR 连接（在调用之前需手动运行 ib.disconnect()）
ib.disconnect()


In [ ]:
#Code3final	股票操作
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from scipy.optimize import minimize
from ib_insync import IB, Stock, MarketOrder

# 初始化 IBKR 连接
ib = IB()
client_id = random.randint(1, 1000)
ib.connect('127.0.0.1', 7497, clientId=client_id)  # 确保连接到 IBKR PRO 纸面交易账户

# 设置市场数据类型为延迟数据
ib.reqMarketDataType(3)  # 使用延迟市场数据

# 取消所有未完成的订单
def cancel_all_open_orders():
    open_orders = ib.openOrders()
    for order in open_orders:
        ib.cancelOrder(order)
    ib.sleep(1)  # 等待取消操作完成
    print("所有未完成的订单已取消。")

# 在连接之后立即取消所有未完成的订单
cancel_all_open_orders()

# 获取每只股票的最近一个月数据并计算年化预期收益和协方差
def fetch_monthly_data(buy_signals):
    returns = []
    stock_data = {}

    for symbol in buy_signals:
        stock = Stock(symbol, 'SMART', 'USD')
        bars = ib.reqHistoricalData(stock, '', '1 M', '1 day', 'TRADES', useRTH=True)
        df = pd.DataFrame([{'date': bar.date, 'close': bar.close} for bar in bars])
        df.set_index('date', inplace=True)

        df['returns'] = df['close'].pct_change()
        annual_return = df['returns'].mean() * 252  # 年化收益率
        returns.append(annual_return)
        stock_data[symbol] = df['returns'].dropna()

    returns_df = pd.DataFrame(stock_data)
    cov_matrix = returns_df.cov() * 252  # 年化协方差矩阵

    return returns, cov_matrix

# 计算组合的负夏普比率
def calculate_portfolio_sharpe(weights, returns, cov_matrix, risk_free_rate=0.01):
    portfolio_return = np.dot(weights, returns)
    portfolio_stddev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_stddev
    return -sharpe_ratio  # 取负值以便于最小化

# 优化组合权重
def optimize_portfolio(buy_signals, budget):
    returns, cov_matrix = fetch_monthly_data(buy_signals)

    num_stocks = len(buy_signals)
    init_weights = np.array([1.0 / num_stocks] * num_stocks)
    constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}
    bounds = tuple((0, 1) for _ in range(num_stocks))

    optimized = minimize(
        calculate_portfolio_sharpe,
        init_weights,
        args=(returns, cov_matrix),
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )

    optimal_weights = optimized.x
    stock_budget_allocation = optimal_weights * budget
    allocation_dict = {symbol: stock_budget_allocation[i] for i, symbol in enumerate(buy_signals)}
    max_sharpe_ratio = -optimized.fun

    return allocation_dict, max_sharpe_ratio

# 执行交易的函数
def execute_trades(allocation):
    trade_report = {}
    
    for symbol, allocated_budget in allocation.items():
        stock = Stock(symbol, 'SMART', 'USD')
        
        # 使用延迟数据请求市场价格
        ib.reqMarketDataType(3)  # 设置为延迟市场数据类型
        market_data = ib.reqMktData(stock, '', False)
        ib.sleep(2)  # 等待数据加载

        # 检查市场数据是否可用
        if market_data.last is None or np.isnan(market_data.last):
            print(f"{symbol} 市场数据不可用，跳过。")
            continue

        # 检查预算是否足够购买至少一股
        if allocated_budget < market_data.last:
            print(f"{symbol} 的预算不足以购买至少一股，跳过。")
            continue

        # 计算购买的数量，确保至少购买一股
        quantity = max(1, int(allocated_budget // market_data.last))

        # 模拟下单操作
        print(f"下单: BUY {quantity} 股 {symbol} at ${market_data.last:.2f}")
        
        # 添加到交易报告
        trade_report[symbol] = {
            "Action": "BUY",
            "Quantity": quantity,
            "Price": market_data.last,
            "Status": "PendingSubmit"  # 这是模拟的状态
        }

    return trade_report




# 主流程
def main():
    buy_signals = generate_signals()  # 假设这些是买入信号的股票
    budget = 100000  # 总预算

    allocation, max_sharpe = optimize_portfolio(buy_signals, budget)
    print("\n优化后的股票分配预算：")
    for symbol, allocated_budget in allocation.items():
        print(f"{symbol} 的分配预算: ${allocated_budget:.2f}")
    print(f"最大化的组合夏普比率: {max_sharpe:.4f}")

    trade_report = execute_trades(allocation)
    print("\n交易报告：")
    for symbol, report in trade_report.items():
        print(f"{symbol}: 买入 {report['Quantity']} 股 at ${report['Price']:.2f}, 状态: {report['Status']}")

# 运行主程序
main()

# 断开 IBKR 连接
ib.disconnect()


In [ ]:
#Code4final	后续仓位监测和每月调整
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from ib_insync import IB, Stock, MarketOrder
import random
import time

# 初始化 IBKR API 连接
ib = IB()
client_id = random.randint(1, 1000)
ib.connect('127.0.0.1', 7497, clientId=client_id)

# 设置市场数据类型为延迟数据
ib.reqMarketDataType(3)

# 1. 批量获取市场数据
def get_batch_market_data(symbols):
    market_data = {}
    for symbol in symbols:
        stock = Stock(symbol, 'SMART', 'USD')
        ib.qualifyContracts(stock)
        data = ib.reqMktData(stock, '', False, False)
        ib.sleep(2)  # 等待数据加载

        # 检查是否成功获取数据
        if data.last is not None:
            market_data[symbol] = {
                'Last Price': data.last,
                'Bid Price': data.bid,
                'Ask Price': data.ask
            }
            print(f"{symbol}: Last Price: {data.last}, Bid Price: {data.bid}, Ask Price: {data.ask}")
        else:
            print(f"{symbol} 市场数据不可用，跳过。")
    return market_data

# 2. 批量检查持仓的止损
def check_stop_loss(positions, stop_loss=0.25):
    stop_orders = []
    for position in positions:
        symbol = position.contract.symbol
        avg_price = position.averageCost
        current_price = ib.reqMktData(position.contract).last
        if current_price is None:
            continue
        drop_pct = (avg_price - current_price) / avg_price
        if drop_pct >= stop_loss:
            stop_orders.append((symbol, position.position))
    return stop_orders

# 3. 批量计算夏普比率和VaR（优化组合分配）
def calculate_sharpe_var(df):
    daily_returns = df['close'].pct_change().dropna()
    mean_return = daily_returns.mean()
    std_dev = daily_returns.std()
    sharpe_ratio = mean_return / std_dev if std_dev != 0 else np.nan
    var_95 = np.percentile(daily_returns, 5)  # 使用95% VaR
    return sharpe_ratio, var_95

# 4. 执行批量下单
def execute_batch_orders(allocation, market_data):
    trade_report = {}
    for symbol, allocated_budget in allocation.items():
        data = market_data.get(symbol)
        if not data or not data['Ask Price']:  # 检查是否有 ask 价数据
            print(f"{symbol} 市场数据不可用，跳过。")
            continue
        quantity = max(1, int(allocated_budget / data['Ask Price']))
        place_order(symbol, 'BUY', quantity, price=data['Ask Price'])
        trade_report[symbol] = {
            'Quantity': quantity,
            'Price': data['Ask Price'],
            'Status': 'PendingSubmit'
        }
    return trade_report

# 5. 执行单笔下单操作
def place_order(symbol, action, quantity, price=None):
    stock = Stock(symbol, 'SMART', 'USD')
    order = MarketOrder(action, quantity)
    if price:
        order.lmtPrice = price
    trade = ib.placeOrder(stock, order)
    print(f"{action} {quantity} 股 {symbol} at {price if price else '市场价'}, 状态: {trade.orderStatus.status}")

# 6. 每月重新平衡仓位
def monthly_rebalance(budget=100000):  # 假设总预算为$100,000
    symbols = ['AAPL', 'GOOGL', 'MSFT', 'TSLA']  # 示例股票列表
    market_data = get_batch_market_data(symbols)
    
    # 示例分配逻辑（这里只是占位，实际中会计算优化的组合）
    allocation = {symbol: budget / len(market_data) for symbol in market_data.keys()}
    
    # 执行批量下单
    trade_report = execute_batch_orders(allocation, market_data)
    return trade_report

# 主程序入口
def main():
    print(f"执行组合再平衡，当前时间: {datetime.now()}")
    trade_report = monthly_rebalance()
    print("\n交易报告：")
    for symbol, report in trade_report.items():
        print(f"{symbol}: 买入 {report['Quantity']} 股 at ${report['Price']:.2f}, 状态: {report['Status']}")

# 运行主程序
main()

# 断开 IBKR 连接
ib.disconnect()


In [ ]:
#Code5final	损益评估算法
#获取 Benchmark 数据
def fetch_benchmark_data(benchmark='SPX', days=180):
    """
    获取 Benchmark 的历史数据。
    :param benchmark: 基准指数（默认 S&P 500）
    :param days: 获取的天数
    :return: DataFrame 格式的基准数据
    """
    index = Index(benchmark, 'SMART')
    bars = ib.reqHistoricalData(index, '', f'{days} D', '1 day', 'TRADES', useRTH=True)
    df = util.df(bars)
    df['returns'] = df['close'].pct_change().dropna()  # 计算每日收益率
    return df[['date', 'close', 'returns']]

#计算投资组合的收益
def calculate_portfolio_returns_with_cost(positions, start_date, end_date):
    """
    计算投资组合的累计收益，并计入交易费用。
    :param positions: 当前持仓的股票列表
    :param start_date: 起始日期
    :param end_date: 结束日期
    :return: 包含累计收益和费用的 DataFrame
    """
    portfolio_returns = []
    total_cost = 0

    for position in positions:
        symbol = position.contract.symbol
        stock = Stock(symbol, 'SMART', 'USD')

        # 获取历史数据并计算每日收益率
        bars = ib.reqHistoricalData(stock, start_date, end_date, '1 day', 'TRADES', useRTH=True)
        df = util.df(bars)
        df['returns'] = df['close'].pct_change().dropna() * position.position

        # 计算交易费用
        trade_cost = calculate_trade_cost(position.position)
        total_cost += trade_cost  # 累加总费用

        # 存储每只股票的收益
        portfolio_returns.append(df[['date', 'returns']])

    # 合并所有股票收益并计算累计收益
    portfolio_df = pd.concat(portfolio_returns).groupby('date').sum().reset_index()
    portfolio_df['cumulative_return'] = (1 + portfolio_df['returns']).cumprod() - 1

    print(f"总交易费用：{total_cost:.2f} USD")
    return portfolio_df, total_cost

#对比投资组合与 Benchmark 的收益
def compare_with_benchmark_with_cost(portfolio_df, benchmark_df, total_cost):
    """
    比较投资组合与 Benchmark 的收益，并计入交易费用。
    :param portfolio_df: 投资组合的收益数据
    :param benchmark_df: 基准指数的收益数据
    :param total_cost: 总交易费用
    """
    combined_df = pd.merge(portfolio_df, benchmark_df, on='date', suffixes=('_portfolio', '_benchmark'))

    # 调整组合的最终收益，扣除交易费用
    final_portfolio_return = combined_df['cumulative_return_portfolio'].iloc[-1] - (total_cost / 10000)

    # 可视化收益曲线
    plt.figure(figsize=(10, 6))
    plt.plot(combined_df['date'], combined_df['cumulative_return_portfolio'], label='Portfolio', linewidth=2)
    plt.plot(combined_df['date'], combined_df['cumulative_return_benchmark'], label='Benchmark (S&P 500)', linewidth=2)
    plt.xlabel('Date')
    plt.ylabel('Cumulative Return')
    plt.title('Portfolio vs Benchmark Performance (Including Costs)')
    plt.legend()
    plt.grid(True)
    plt.show()

    # 打印最终累计收益
    print(f"组合最终累计收益（扣除费用）：{final_portfolio_return:.2%}")
    print(f"Benchmark 最终累计收益：{combined_df['cumulative_return_benchmark'].iloc[-1]:.2%}")

#集成到主流程
def main():
    symbols = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA']
    budget = 10000  # 总预算

    # 获取当前持仓
    positions = get_current_positions()

    # 获取 Benchmark 数据
    benchmark_df = fetch_benchmark_data(days=180)

    # 计算组合收益并计入交易费用
    start_date = benchmark_df['date'].min().strftime('%Y%m%d')
    end_date = benchmark_df['date'].max().strftime('%Y%m%d')
    portfolio_df, total_cost = calculate_portfolio_returns_with_cost(positions, start_date, end_date)

    # 对比组合与 Benchmark 表现（包含费用）
    compare_with_benchmark_with_cost(portfolio_df, benchmark_df, total_cost)

schedule.every().day.at("17:00").do(main)

while True:
    schedule.run_pending()
    time.sleep(1)
